In [44]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
import pandas as pd
from selenium.webdriver.chrome.options import Options
import traceback


url="https://bcs-express.ru/category/torgovye-rekomendacii"

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response=requests.get(url, headers=headers)

soup=BeautifulSoup(response.text,"html.parser")

print(soup.find_all("div", class_=["TjB6 KSLV Ncpb E6j8","TjB6 KSLV Ncpb E6j8 IhFN"])[0].find("a", class_="iKzE").text) #title
print(soup.find_all("div", class_=["TjB6 KSLV Ncpb E6j8","TjB6 KSLV Ncpb E6j8 IhFN"])[0].find("a", class_="iKzE").attrs['href']) #link


Подтверждаем идею в акциях Мосбиржи. Решение по ставке ЦБ — ключевой драйвер
https://bcs-express.ru/novosti-i-analitika/podtverzhdenie-idei-mosbirzha-reshenie-po-stavke-tsb-v-sentiabre-kliuchevoi-draiver


In [80]:
driver = webdriver.Safari()

driver.get(url)

button = driver.find_element(By.CLASS_NAME,"nlJ0")
button.click()
columns=['title', 'description', 'publishedAt']
df=pd.DataFrame(columns=columns)
k=0
try:
    while True:
            # Прокручиваем вниз
            driver.execute_script("window.scrollBy(0, 500);")

            # Ждем загрузки новых данных
            time.sleep(2)

            soup=BeautifulSoup(driver.page_source,"html.parser")
            page=soup.find_all("div", class_=["TjB6 KSLV Ncpb E6j8","TjB6 KSLV Ncpb E6j8 IhFN"])
            for i in range(k,len(page)):
                  title=page[i].find("a", class_="iKzE").text
                  link=page[i].find("a", class_="iKzE").attrs['href']

                  driver.execute_script(f"window.open('{link}', '_blank');")
                  time.sleep(2)

                  # Переключение на новую вкладку
                  driver.switch_to.window(driver.window_handles[1])
                  description=';'.join(list(map(lambda element: element.text,driver.find_element(By.TAG_NAME, 'body').find_elements(By.TAG_NAME,"p")[:-8])))
                  publish_date=driver.find_element(By.TAG_NAME, 'time').text
                  df.loc[len(df)]=[title,description,publish_date]
                  driver.close()
                  driver.switch_to.window(driver.window_handles[0])
            if len(df)==200:
                  break
                  




except Exception as e:
      print(f"find error:{e}")
      traceback.print_exc()

finally:
      driver.quit()



SessionNotCreatedException: Message: Could not create a session: The automation session ended unexpected while attempting to pair with it.


In [79]:
driver.quit()